In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv', parse_dates=['datetime'])
test = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv', parse_dates=['datetime'])
sub = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')

train.shape, test.shape, sub.shape

In [3]:
sub.columns

In [4]:
import matplotlib.pyplot as plt ## seaborn 보다 고급 시각화 가능. but 코드 복잡
import seaborn as sns           ## seaborn은 matplotlib보다 간단하게 사용 가능

In [5]:
col_names = [ 'season', 'holiday', 'workingday', 'weather'  ]
i = 0
plt.figure(figsize=(12,10))   # 전체 그래프의 크기 지정 

for name in col_names:        # 컬럼명을 전달 리스트 수 만큼 반복 -> 4회 
    i = i + 1                 # 숫자를 1씩 증가.
    plt.subplot(2,2,i)        # 2행 2열에 i번째 그래프 선택
    sns.countplot(x=name, data=train)  # i번째 그래프에 sns.countplot를 그리겠다.
    
plt.show()  # 주피터에서 보여주지만, 다른곳(editor, pycharm)에서는 이걸 실행시켜야 한다.

In [6]:
### temp, atemp, humidity, windspeed

num_names = ['temp', 'atemp', 'humidity', 'windspeed']
train.columns

In [7]:
i = 0
plt.figure(figsize=(12,25))   # 전체 그래프의 크기 지정 (가로, 세로)

for name in num_names:        # 컬럼명을 전달 리스트 수 만큼 반복 -> 4회
    i = i + 1                 # 숫자를 1씩 증가.
    plt.subplot(4,2,i*2-1)        # 2행 2열에 i번째 그래프 선택
    sns.histplot(x=name, data=train)  # i번째 그래프에 sns.histplot를 그리겠다.
    plt.title("train feature")
    
    plt.subplot(4,2,i*2)        # 2행 2열에 i번째 그래프 선택
    sns.histplot(x=name, data=test)  # i번째 그래프에 sns.histplot를 그리겠다.
    plt.title("test feature")
    
plt.show()

In [8]:
new_tr = train.copy()   # 데이터 백업
new_test = test.copy()
new_tr.columns

In [9]:
## 더미변수, 파생변수 생성
new_tr['year'] = new_tr['datetime'].dt.year
new_tr.head()

In [10]:
new_tr['month'] = new_tr['datetime'].dt.month
new_tr['day'] = new_tr['datetime'].dt.day
new_tr['hour'] = new_tr['datetime'].dt.hour
new_tr['minute'] = new_tr['datetime'].dt.minute
new_tr['second'] = new_tr['datetime'].dt.second
new_tr['dayofweek'] = new_tr['datetime'].dt.dayofweek
new_tr.head()

In [11]:
new_test['year'] = new_test['datetime'].dt.year
new_test['month'] = new_test['datetime'].dt.month
new_test['day'] = new_test['datetime'].dt.day
new_test['hour'] = new_test['datetime'].dt.hour
new_test['minute'] = new_test['datetime'].dt.minute
new_test['second'] = new_test['datetime'].dt.second
new_test['dayofweek'] = new_test['datetime'].dt.dayofweek
new_test.head()

In [12]:
datetime_names = ['year', 'month', 'day', 'hour', 'dayofweek', 'second']

i=0
plt.figure(figsize=(15,15))
for name in datetime_names:
    i = i + 1
    plt.subplot(3,2,i)
    sns.barplot(x=name, y='count', data=new_tr)
    
plt.show()

In [13]:
plt.figure(figsize=(15,10))
g = sns.heatmap(new_tr.corr(), annot=True, fmt=".2f", cmap="coolwarm", cbar=False)

# 비선형 변환

In [31]:
# new_tr의 log 적용 값 생성
new_tr['log_count'] = np.log1p(new_tr['count'])

In [38]:
# log 적용 전 후 비교
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
sns.histplot(x='count',data=new_tr)

plt.subplot(1,2,2)
sns.histplot(x='log_count',data=new_tr)

In [45]:
feature_names = [ 'season', 'holiday', 'workingday', 'weather', 
                  'temp', 'atemp', 'humidity', 'windspeed', 
                  "year", "hour", "dayofweek"]  # 공통 변수

X_tr_all = new_tr[feature_names]  # 학습용 데이터 변수 선택
y_tr_all = new_tr["log_count"]        # 렌탈 대수 변수 값 선택

X_test_all = new_test[feature_names]    # 테스트 데이터의 변수 선택

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_tr_all, 
                                                    y_tr_all,
                                                    test_size=0.3,
                                                    random_state=77)

In [48]:
from sklearn.linear_model import LinearRegression  
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor   # 앙상블(의사결정트리 확장판)
from sklearn.ensemble import GradientBoostingRegressor

In [49]:
# 선형회귀 모델 훈련 및 평가
model = LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)

# 결정계수 확인
print("학습용 데이터 결정계수: {:.3f}".format(model.score(X_train, y_train)))
print("테스트 데이터 결정계수: {:.3f}".format(model.score(X_test, y_test)))

mse_val = ((pred-y_test) ** 2).sum() / len(pred)
print('MSE : {:.3f}'.format(mse_val))

In [50]:
# 의사결정트리 모델 훈련 및 평가
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
pred= model.predict(X_test)

# 결정계수 확인
print("학습용 데이터 결정계수: {:.3f}".format(model.score(X_train, y_train)))
print("테스트 데이터 결정계수: {:.3f}".format(model.score(X_test, y_test)))

mse_val = ((pred-y_test) ** 2).sum() / len(pred)
print('MSE : {:.3f}'.format(mse_val))

In [51]:
# 랜덤포레스트 모델 훈련 및 평가
model = RandomForestRegressor()
model.fit(X_train, y_train)
pred= model.predict(X_test)

# 결정계수 확인
print("학습용 데이터 결정계수: {:.3f}".format(model.score(X_train, y_train)))
print("테스트 데이터 결정계수: {:.3f}".format(model.score(X_test, y_test)))

mse_val = ((pred-y_test) ** 2).sum() / len(pred)
print('MSE : {:.3f}'.format(mse_val))

In [52]:
# 그래디언트 부스팅 모델 훈련 및 평가
model = GradientBoostingRegressor()
model.fit(X_train, y_train)
pred= model.predict(X_test)

# 결정계수 확인
print("학습용 데이터 결정계수: {:.3f}".format(model.score(X_train, y_train)))
print("테스트 데이터 결정계수: {:.3f}".format(model.score(X_test, y_test)))

mse_val = ((pred-y_test) ** 2).sum() / len(pred)
print('MSE : {:.3f}'.format(mse_val))

### 최적의 모델 선택
* 랜덤 포레스트 모델

In [22]:
# 랜덤포레스트 모델 훈련 및 평가
for i in range(1,6):
    for d in range(1,5):
        X_train, X_test, y_train, y_test = train_test_split(X_tr_all, 
                                                    y_tr_all,
                                                    test_size=(i/10),
                                                    random_state=77)
    
        model = RandomForestRegressor(n_jobs=-1, max_depth=d, random_state=0)
        model.fit(X_train, y_train)
        pred= model.predict(X_test)

        # 결정계수 확인
        print(f'훈련:테스트 = {10-i}:{i}, max_depth = {d}')
        print("학습용 데이터 결정계수: {:.3f}".format(model.score(X_train, y_train)))
        print("테스트 데이터 결정계수: {:.3f}".format(model.score(X_test, y_test)))

        mse_val = ((pred-y_test) ** 2).sum() / len(pred)
        print('MSE : {:.3f}'.format(mse_val))
        print()

In [23]:
# max_depth : 5-10
for i in range(1,6):
    for d in range(5,11):
        X_train, X_test, y_train, y_test = train_test_split(X_tr_all, 
                                                    y_tr_all,
                                                    test_size=(i/10),
                                                    random_state=77)
    
        model = RandomForestRegressor(n_jobs=-1, max_depth=d, random_state=0)
        model.fit(X_train, y_train)
        pred= model.predict(X_test)

        # 결정계수 확인
        print(f'훈련:테스트 = {10-i}:{i}, max_depth = {d}')
        print("학습용 데이터 결정계수: {:.3f}".format(model.score(X_train, y_train)))
        print("테스트 데이터 결정계수: {:.3f}".format(model.score(X_test, y_test)))

        mse_val = ((pred-y_test) ** 2).sum() / len(pred)
        print('MSE : {:.3f}'.format(mse_val))
        print()

In [24]:
# test_size => 9:1, 8:2
# max_depth => 8이상 8-13
for i in range(1,3):
    for d in range(8,14):
        X_train, X_test, y_train, y_test = train_test_split(X_tr_all, 
                                                    y_tr_all,
                                                    test_size=(i/10),
                                                    random_state=77)
    
        model = RandomForestRegressor(n_jobs=-1, max_depth=d, random_state=0)
        model.fit(X_train, y_train)
        pred= model.predict(X_test)

        # 결정계수 확인
        print(f'훈련:테스트 = {10-i}:{i}, max_depth = {d}')
        print("학습용 데이터 결정계수: {:.3f}".format(model.score(X_train, y_train)))
        print("테스트 데이터 결정계수: {:.3f}".format(model.score(X_test, y_test)))

        mse_val = ((pred-y_test) ** 2).sum() / len(pred)
        print('MSE : {:.3f}'.format(mse_val))
        print()

In [25]:
# test_size => 9:1, 8:2
# max_depth => 기본
for i in range(1,3):
    X_train, X_test, y_train, y_test = train_test_split(X_tr_all, 
                                                        y_tr_all,
                                                        test_size=(i/10),
                                                        random_state=77)
    
    model = RandomForestRegressor(n_jobs=-1, random_state=0)
    model.fit(X_train, y_train)
    pred= model.predict(X_test)

    # 결정계수 확인
    print(f'훈련:테스트 = {10-i}:{i}')
    print("학습용 데이터 결정계수: {:.3f}".format(model.score(X_train, y_train)))
    print("테스트 데이터 결정계수: {:.3f}".format(model.score(X_test, y_test)))

    mse_val = ((pred-y_test) ** 2).sum() / len(pred)
    print('MSE : {:.3f}'.format(mse_val))
    print()

In [53]:
# test_size => 9:1
# max_depth => 기본


X_train, X_test, y_train, y_test = train_test_split(X_tr_all, 
                                                    y_tr_all,
                                                    test_size=0.1,
                                                    random_state=77)

model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(X_train, y_train)
pred= model.predict(X_test)

# 결정계수 확인
print(f'훈련:테스트 = 9:1')
print("학습용 데이터 결정계수: {:.3f}".format(model.score(X_train, y_train)))
print("테스트 데이터 결정계수: {:.3f}".format(model.score(X_test, y_test)))

mse_val = ((pred-y_test) ** 2).sum() / len(pred)
print('MSE : {:.3f}'.format(mse_val))
print()

In [54]:
pred = model.predict(X_test_all)   # 예측
sub['count'] = np.expm1(pred) # 자연로그를 원상복귀해 준 뒤 -1 해주는 함수
sub.loc[sub['count'] < 0, 'count'] = 0
sub.head(3)

In [55]:
# 처음 만는 제출용 csv 파일, 행번호를 없애기
sub.to_csv("sub03_log.csv", index=False)

In [28]:
from sklearn.model_selection import cross_val_score

In [29]:
# test_size => 9:1
# max_depth => 기본

X_train, X_test, y_train, y_test = train_test_split(X_tr_all, 
                                                    y_tr_all,
                                                    test_size=0.1,
                                                    random_state=77)

model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(X_train, y_train)
pred= model.predict(X_test)

# 결정계수 확인
print(f'훈련:테스트 = 9:1')
print('교차검증 수 = 3')
tr_score = cross_val_score(model, X_train, y_train, cv=3).mean()
test_score = cross_val_score(model, X_test, y_test, cv=3).mean()
print("학습용 데이터 결정계수: {:.3f}".format(tr_score))
print("테스트 데이터 결정계수: {:.3f}".format(test_score))

mse_val = ((pred-y_test) ** 2).sum() / len(pred)
print('MSE : {:.3f}'.format(mse_val))
print()

In [30]:
# col : 'year', 'month', 'day', 'hour', 'dayofweek', 'humidity', 'temp', 'atemp', 'weather', 'season'
X_tr_all.columns

## 다른 모델

In [58]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor

In [56]:
from sklearn.model_selection import cross_val_score

In [57]:
def model_val(model_name):
    model = model_name
    model.fit(X_train, y_train)
    scores = cross_val_score(model, X_test, y_test, cv=5, scoring='neg_mean_squared_error')
    
    print("MSE : ", scores)
    avg_score = np.abs(scores.mean())
    print("MSE 평균 :", avg_score)
    return avg_score